# Consensus Protocol Deep Dive

## Understanding PBFT for Financial Trading

This notebook explores the custom Practical Byzantine Fault Tolerant (PBFT) consensus algorithm optimized for financial transactions.

## Why PBFT?

### Requirements for Trading Systems

1. **Safety**: Transactions cannot be reversed once confirmed
2. **Liveness**: System must continue operating despite failures
3. **Fairness**: Prevents front-running and order manipulation
4. **Performance**: Sub-2-second finality
5. **Byzantine Tolerance**: Works with < 33% malicious nodes

In [ ]:
# PBFT Phases Visualization
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

fig, ax = plt.subplots(figsize=(14, 6))

phases = [
    {'name': 'Pre-prepare', 'x': 1, 'color': '#FF6B6B'},
    {'name': 'Prepare', 'x': 4, 'color': '#4ECDC4'},
    {'name': 'Commit', 'x': 7, 'color': '#45B7D1'},
    {'name': 'Reply', 'x': 10, 'color': '#96CEB4'}
]

for phase in phases:
    ax.add_patch(mpatches.FancyBboxPatch((phase['x']-0.8, 0), 1.6, 2, 
                                         boxstyle='round,pad=0.1', 
                                         facecolor=phase['color'], alpha=0.7))
    ax.text(phase['x'], 1, phase['name'], ha='center', va='center', 
           fontsize=12, fontweight='bold')

# Draw arrows
for i in range(len(phases)-1):
    ax.annotate('', xy=(phases[i+1]['x']-0.9, 1), xytext=(phases[i]['x']+0.9, 1),
               arrowprops=dict(arrowstyle='->', lw=2))

ax.set_xlim(-0.5, 12)
ax.set_ylim(-0.5, 2.5)
ax.axis('off')
ax.set_title('PBFT Consensus Phases', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

## Phase 1: Pre-prepare

```
Primary node p:
  1. Collects pending orders from mempool
  2. Creates block B with sequence number s
  3. Computes digest D = hash(B)
  4. Broadcasts: PRE-PREPARE <s, D> to all backups
```

**Duration**: ~200ms
**Validators**: 1 (primary only)

## Phase 2: Prepare

```
Backup nodes:
  1. Verify PRE-PREPARE message
  2. Check block contents and signature
  3. Broadcast: PREPARE <s, D> to all nodes
  4. Track prepare votes
```

**Quorum**: Need 2f + 1 prepares (f = max faulty nodes)
**Duration**: ~300ms

## Phase 3: Commit

```
All nodes (once 2f+1 prepares received):
  1. Broadcast: COMMIT <s, D>
  2. Apply block locally
  3. Execute transactions
  4. Update order book
```

**Safety**: Block is finalized (irreversible)
**Duration**: ~400ms

## Example: Order Matching Consensus

**Scenario**: 4 consensus nodes (N=4, f=1)

```
Node 0 (Leader): Proposes Block 100 with 50 trades
  |
  +-> Broadcast PRE-PREPARE

Nodes 1, 2, 3: Receive and validate
  |
  +-> Each broadcasts PREPARE

Consensus reached: 4 PREPARE messages (> 2f+1 = 3)
  |
  +-> All nodes commit
  +-> Trades executed
  +-> Order book updated
  +-> Settlement recorded
```

## Optimization for Trading

### Batching
- Accumulate 100-1000 orders per block
- Reduces consensus overhead
- Improves throughput (target: 10K TPS)

### Priority
- High-value trades prioritized
- Emergency liquidation orders fast-tracked
- Market-making orders get fair treatment

### View Change
- If primary fails, automatic leader election
- New leader takes over in ~2 seconds
- No trades lost

In [ ]:
# Consensus Metrics
metrics = {
    'Throughput': '10,000 TPS',
    'Finality': '<2 seconds',
    'Byzantine Tolerance': '< 33% malicious nodes',
    'Block Time': '2 seconds',
    'Message Complexity': 'O(n²) per block',
    'Safety': 'Proven (BFT guarantee)',
    'Liveness': 'Guaranteed under sync conditions'
}

for metric, value in metrics.items():
    print(f'{metric:30} : {value}')